# Assignment: Finite Element Method (1D)

In this assignment, you are asked to use and extend the code from the preparation notebook on the Finite Element Method for one-dimensional problems. Before making the assignment, study the preparation notebook.

## A modification to the PDE: continuous elastic support

<p align="center">
<img src="https://raw.githubusercontent.com/fmeer/public-files/main/AssignmentC1.PNG" width="600"/>
</p>

For this exercise we consider the same bar as in the preparation notebook. However, now the bar is elastically supported. An example of this would be a foundation pile in soil. The continuous spring constant is $k=100$ kN/m$^2$, all other parameters stay the same.

Looking into the physics of this problem, we can describe it using the following differential equation:

$$ -EA \frac{\partial^2 u}{\partial x^2} + ku = q $$

The additional term with respect to the case without elastic support is the second term on the left hand side: $ku$. 

The finite element discretized version of this PDE can be obtained following the same steps as shown for the unsupported bar on the preparation notebook. Note that there are no derivatives in the $ku$ which means that integration by parts does not need to be applied on this term. The following expression is found for the discretized form:

$$\left[\int \mathbf{B}^T EA \mathbf{B} + \mathbf{N}^T k \mathbf{N} \,dx\right]\mathbf{u} = \int \mathbf{N}^T q \,d x + \mathbf{N}^T h \Bigg|_{x=L} $$

The only alteration from the procedure as implemented in the preparation notebook is the formulation of the stiffness matrix, which now consists of two terms:

$$ \mathbf{K} = \int \mathbf{B}^TEA\mathbf{B} + \mathbf{N}^Tk\mathbf{N}\,dx $$

Considering the use of linear shape functions, with $\tilde{x}$ as the local coordinate, and by substitution we get:

$$\mathbf{N} = \left[\begin{matrix}1-\frac{\tilde{x}}{\Delta x} & \frac{\tilde{x}}{\Delta x}\end{matrix}\right]$$

$$\mathbf{B} = \left[\begin{matrix}\frac{-1}{\Delta x} & \frac{1}{\Delta x}\end{matrix}\right]$$

$$\mathbf{K_e} = \int \left[\begin{matrix}\frac{-1}{\Delta x} \\ \frac{1}{\Delta x}\end{matrix}\right] EA \left[\begin{matrix}\frac{-1}{\Delta x} & \frac{1}{\Delta x}\end{matrix}\right] + \left[\begin{matrix}1-\frac{\tilde{x}}{\Delta x} \\ \frac{\tilde{x}}{\Delta x}\end{matrix}\right] k \left[\begin{matrix}1-\frac{\tilde{x}}{\Delta x} & \frac{\tilde{x}}{\Delta x}\end{matrix}\right]\,dx$$

$$\mathbf{K_e} = \int \frac{EA}{\Delta x}\left[\begin{matrix}1 & -1 \\ -1 & 1\end{matrix}\right] + k \left[\begin{matrix}1-2\frac{\tilde{x}}{\Delta x} + \frac{\tilde{x}^2}{\Delta x^2} & \frac{\tilde{x}}{\Delta x} - \frac{\tilde{x}^2}{\Delta x^2} \\ \frac{\tilde{x}}{\Delta x} - \frac{\tilde{x}^2}{\Delta x^2} & \frac{\tilde{x}^2}{\Delta x^2}\end{matrix}\right]\,dx$$

As becomes clear, the second term in the integral of the element matrix depends on $x$. The introduction of a continuous spring delivers a quadratic term in the integral. Therefore it is not possible to calculate the integral by just taking the value in the middle and multiplying it by the length of the element. In this case we need to loop over a set of integration points to get an good approximation. 

$$ \mathbf{K_e} = \sum_{i=1}^2 \left(\mathbf{B}^T(\tilde{x}_i)EA\mathbf{B}(\tilde{x}_i) + \mathbf{N}^T(\tilde{x}_i) k\mathbf{N}(\tilde{x}_i) \right) w_i$$

Using Gauss integration we can integrate the element stiffness matrix exactly with two points as:

$$ \tilde{x}_1 = \frac{\Delta x}{2} - \frac{\Delta x}{2\sqrt{3}}$$

$$ \tilde{x}_2 = \frac{\Delta x}{2} + \frac{\Delta x}{2\sqrt{3}}$$

$$ w_1 = w_2 = \frac{\Delta x}{2}$$

## Exercise 1: Code implementation

As discussed, the only change needed is the calculation of the local stiffness matrix. The formulas shown above show the change needed in the numerical integration. Apply these changes to the code below to model the described problem.

Remarks:
- A function to evaluate the vector $\mathbf{N}$ at a given location has already been coded in the block below
- The `get_element_matrix` function already includes a loop over two integration points as needed for the new term in the stiffness matrix

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Returns the evaluated N vector
# - The local coordinate of evaluation "x_local"
# - The element size "dx"
def evaluate_N(x_local, dx):
    return [1-x_local/dx, x_local/dx]
    

# Returning the stiffness matrix of a single element
# - The node to the left of the element "n_left"
# - The node to the right of the element "n_right"
# - The total number of nodes/DOFs in the system "n_DOF"
# - The stiffness of the bar "EA"
# - The length of an element "dx"
def get_element_matrix(n_left, n_right, n_DOF, EA, dx):
    # Creating a matrix with the required size
    B = np.zeros((1,n_DOF))    # Defining in 2 dimensions is nessecary for transpose
    B[0,n_left] = -1/dx
    B[0,n_right] = 1/dx    
    
    # Defining integration locations and weights
    integration_locations = [(dx - dx/(3**0.5))/2, (dx + dx/(3**0.5))/2]
    integration_weights = [dx/2, dx/2]
    
    # Setting up the local elementstiffness matrix
    K_loc = np.zeros((n_DOF, n_DOF))
    
    for i in range(2):
        K_loc += np.dot(np.transpose(B), EA*B)*integration_weights[i]

    return K_loc

In [ ]:
# Returns the global K matrix of a bar (EA and dx not included)
# - The length of the bar "bar_length"
# - The number of elements "n_el"
# - The stiffness of the bar "EA"
def assemble_global_K_bar(bar_length, n_el, EA):
    n_DOF = n_el+1
    dx = bar_length/n_el
    # Creating a zero-filled matrix which will be filled with the local contributions
    K_global = np.zeros((n_DOF, n_DOF))
    
    for i in range(n_el):
        # Get the contribution of the element which lies between node i and node i+1
        K_global += get_element_matrix(i, i+1, n_DOF, EA, dx)
    
    return K_global

# Returns the global f vector with continous forces on a bar
# - The length of the bar "bar_length"
# - The number of elements "n_el"
# - The force on the bar "EA"
def assemble_global_f_bar(bar_length, n_el, q):
    n_DOF = n_el+1
    dx = bar_length/n_el
    # Creating a zero-filled vector which will be filled with the local contributions
    f_global = np.zeros((n_DOF))
    
    for i in range(n_el):
        # Get the contribution of the element which lies between node i and node i+1
        f_global[i] += (q*dx)/2
        f_global[i+1] += (q*dx)/2
    
    return f_global

In [ ]:
length = 3          # Length in meters (m)
EA = 1e3            # Stiffness EA (kN)
n_element = 30      # Number of elements
n_node = 31         # Number of nodes
F_right = 10        # Load applied at the right boundary
u_left = 0          # Displacement at the left boundary
q_load = -5        #distibuted load (kN/m)

dx = length/n_element
x = np.linspace(0,length,n_node)

K = assemble_global_K_bar(length, n_element, EA)
K_inv = np.linalg.inv(K[1:n_node, 1:n_node])

# Assemble f with the continuous forces
f = assemble_global_f_bar(length, n_element, q_load)

# Add boundary conditions to f
f[n_node-1] += F_right

# Initialize vector u with displacements
u = np.zeros(n_node)

# Solve the system
u[1:n_node] = np.dot(K_inv, f[1:n_node])

# Plot the result
plt.plot(x,u)

## Exercise 2: Derive the discretized form

Derive the discretized form of the PDE given above. You can follow the same steps as in the preparation notebook for the term with $EA$ and the right hand side, but now with the additional term $ku$ in the PDE. Show that this term leads to the $\int\mathbf{N}^Tk\mathbf{N}\,dx$ term in the stiffness matrix.


## Exercise 3: Numerical integration (Optional)

In the `get_element_matrix` function, the integration scheme for numerical integration over the element domain is already given. How can the weights and locations used in the code be derived from standard values given for Gauss integration?